In [ ]:
# This is an example snippet!
# To create your own, add a new snippet block to the
# snippets.json file in your jupyter nbextensions directory:
# /nbextensions/snippets/snippets.json
import this

In [1]:
import pandas as pd
import random
import itertools

Downloaded data from: http://web.mta.info/developers/turnstile.html

Documentations at: http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description.txt

Map of the MTA system: http://web.mta.info/maps/submap.html

In [2]:
df = pd.read_csv('turnstile_May26.txt')

In [3]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,00:00:00,REGULAR,6625470,2245312
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,04:00:00,REGULAR,6625494,2245314
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,08:00:00,REGULAR,6625514,2245338
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,12:00:00,REGULAR,6625595,2245415
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,16:00:00,REGULAR,6625791,2245481


Fix 'EXITS' column name (it has many spaces at the end of it)

In [4]:
# old_exit_name = df.columns[-1]
df.rename(columns={df.columns[-1]: 'EXITS'}, inplace = True)

In [5]:
backup_df = df.copy()

In [6]:
df['DATETIME'] = pd.to_datetime(df.DATE + ' ' + df.TIME)

In [44]:
df['ENTRY_DIFFS'] = df['ENTRIES'].diff()

In [46]:
df.head(21)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,ENTRY_DIFFS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,00:00:00,REGULAR,6625470,2245312,2018-05-19 00:00:00,NaN
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,04:00:00,REGULAR,6625494,2245314,2018-05-19 04:00:00,24.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,08:00:00,REGULAR,6625514,2245338,2018-05-19 08:00:00,20.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,12:00:00,REGULAR,6625595,2245415,2018-05-19 12:00:00,81.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,16:00:00,REGULAR,6625791,2245481,2018-05-19 16:00:00,196.0
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/19/2018,20:00:00,REGULAR,6626018,2245520,2018-05-19 20:00:00,227.0
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/20/2018,00:00:00,REGULAR,6626130,2245537,2018-05-20 00:00:00,112.0
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/20/2018,04:00:00,REGULAR,6626149,2245545,2018-05-20 04:00:00,19.0
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/20/2018,08:00:00,REGULAR,6626160,2245564,2018-05-20 08:00:00,11.0
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/20/2018,12:00:00,REGULAR,6626223,2245620,2018-05-20 12:00:00,63.0


In [ ]:
df['STATION_KEY'] = df

Make list of columns which we'll use at the key (i.e. the first 4) and a list of the rest.

In [8]:
# key_cols = list(df.columns[:4])

['C/A', 'UNIT', 'SCP', 'STATION']

In [9]:
# other_cols = list(df.columns[4:])

['LINENAME',
 'DIVISION',
 'DATE',
 'TIME',
 'DESC',
 'ENTRIES',
 'EXITS',
 'DATETIME']

Create dictionary, using 4-column key:

In [ ]:
grp = df.groupby()

In [48]:
grp1 = df.groupby(key_cols + ['DATE'])['ENTRY_DIFFS'].sum().reset_index()

In [49]:
grp1.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRY_DIFFS
0,A002,R051,02-00-00,59 ST,05/19/2018,548.0
1,A002,R051,02-00-00,59 ST,05/20/2018,572.0
2,A002,R051,02-00-00,59 ST,05/21/2018,1298.0
3,A002,R051,02-00-00,59 ST,05/22/2018,1448.0
4,A002,R051,02-00-00,59 ST,05/23/2018,1435.0


In [39]:
import datetime as dt

In [40]:
grp2 = df.groupby(key_cols + ['DATETIME'].dt.date())['ENTRIES'].sum().reset_index()

AttributeError: 'list' object has no attribute 'dt'